<a href="https://colab.research.google.com/github/GAUTAMMANU/projminor/blob/main/code/modelling-Brazil-new2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Settings

In [42]:
# !git clone https://github.com/GAUTAMMANU/projminor.git
# %cd projminor

In [43]:
# %cd code
# insert your desired path to work on
import os
from os.path import join
project_path = os.path.dirname(os.getcwd())
# os.chdir(join('..','data'))
os.getcwd()

'e:\\minor\\proj\\code'

Run the following section one time per session. This cell links the code folder to the python exectution path.

In [44]:
import sys
sys.path.append(join(project_path, 'code'))

The following cell allows Jupyter Notebooks to detect changes in external code and to automatically update it without restarting the runtime.

In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Plots settings.

In [46]:
import matplotlib
font = {'family':'Arial', 'size':'15', 'weight':'normal'}

matplotlib.rc('font', **font)

Set folder structure.

In [47]:
config = {
    'main_brazil': 'Brazil',
    'main_peru': 'Peru',
    'baseline': join(project_path, "baseline_models"),
    'output': join(project_path, "code", "saved_models"),
    'metrics': join(project_path, "code", "metrics")
}
project_path

# List comprehension for the folder structure code
[os.makedirs(val, exist_ok=True) for key, val in config.items()]

[None, None, None, None, None]

# **AI4Dengue forecasting**
![](https://drive.google.com/uc?export=view&id=1J5Bt5Cks-e2IV-dEJLHJkuwXFJNFAZgr)

In [48]:
import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from glob import glob
from config import DEP_NAMES, GROUPED_VARS, DATA_REDUCER_SETTINGS, DATA_PROCESSING_SETTINGS


# Data

## Load the dataframe
**This dataframe comprises all the variables (climatic, epidemiological etc.) acquired for each Department during a defined number of years.**

In [49]:
dataframe = pd.read_csv(join('dataset', "Brazil_UF_dengue_monthly.csv"))
dataframe.head()
dataframe.iloc[1000]

Date                2004-02-01
Year                      2004
Month                        2
CD_UF                       12
area_km2            164173.431
                       ...    
rdpc_def_vulner         119.68
t_analf_18m              17.79
t_formal_18m             46.45
t_fundc_ocup18m           55.2
t_medioc_ocup18m         39.61
Name: 1000, Length: 62, dtype: object

**Load CNN results as columns to dataframe.**

In [50]:
cnn = pd.read_csv(join('saved_models', "cnn_dataframe.csv")).drop('Unnamed: 0', axis=1)
cnn['CD_UF'] = cnn['CD_UF'].astype(np.int64)

assert dataframe.shape[0] == cnn.shape[0]
assert all(dataframe['CD_UF'].unique() == cnn['CD_UF'].unique())
cnn

,CD_UF,CNN_all,CNN_0-19
0,11,1.000000,1.000000
1,11,1.000000,1.000000
2,11,1.000000,1.000000
3,11,1.000000,1.000000
4,11,32.159859,17.186546
...,...,...,...
6151,53,29.022461,16.795465
6152,53,20.277210,5.658783
6153,53,7.219064,16.862005
6154,53,17.866333,28.619926


In [51]:
dataframe.sort_values(['CD_UF', 'Date'], inplace=True, ignore_index=True)
dataframe

,Date,Year,Month,CD_UF,area_km2,NDVI_d,dewpoint_temperature_2m_d,humidity_d,max_temperature_2m_d,min_temperature_2m_d,...,pea10a14,pea15a17,pea18m,t_eletrica,t_densidadem2,rdpc_def_vulner,t_analf_18m,t_formal_18m,t_fundc_ocup18m,t_medioc_ocup18m
0,2001-01-01,2001,1,11,237765.347,0.154301,295.674980,88.460308,303.987216,294.155015,...,18698,34904,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93
1,2001-02-01,2001,2,11,237765.347,0.216873,295.944060,88.856948,304.738755,294.332566,...,18698,34904,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93
2,2001-03-01,2001,3,11,237765.347,0.239112,296.092747,89.305463,304.620829,294.304126,...,18698,34904,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93
3,2001-04-01,2001,4,11,237765.347,0.334660,296.186143,88.590375,304.168669,293.921815,...,18698,34904,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93
4,2001-05-01,2001,5,11,237765.347,0.378931,295.562972,86.939606,303.903043,293.395959,...,18698,34904,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,2019-08-01,2019,8,53,5760.784,0.362744,282.150351,42.202163,304.210083,287.271135,...,10706,36652,1361053,99.91,23.48,171.62,3.66,71.62,76.39,61.00
6152,2019-09-01,2019,9,53,5760.784,0.317748,281.820936,34.023500,307.566780,290.719267,...,10706,36652,1361053,99.91,23.48,171.62,3.66,71.62,76.39,61.00
6153,2019-10-01,2019,10,53,5760.784,0.271795,286.196146,45.486547,307.716003,291.720099,...,10706,36652,1361053,99.91,23.48,171.62,3.66,71.62,76.39,61.00
6154,2019-11-01,2019,11,53,5760.784,0.235493,290.445969,64.916154,306.706715,291.496597,...,10706,36652,1361053,99.91,23.48,171.62,3.66,71.62,76.39,61.00


In [52]:
dataframe = pd.concat([dataframe, cnn[['CNN_all', 'CNN_0-19']]], axis=1)
dataframe

,Date,Year,Month,CD_UF,area_km2,NDVI_d,dewpoint_temperature_2m_d,humidity_d,max_temperature_2m_d,min_temperature_2m_d,...,pea18m,t_eletrica,t_densidadem2,rdpc_def_vulner,t_analf_18m,t_formal_18m,t_fundc_ocup18m,t_medioc_ocup18m,CNN_all,CNN_0-19
0,2001-01-01,2001,1,11,237765.347,0.154301,295.674980,88.460308,303.987216,294.155015,...,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93,1.000000,1.000000
1,2001-02-01,2001,2,11,237765.347,0.216873,295.944060,88.856948,304.738755,294.332566,...,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93,1.000000,1.000000
2,2001-03-01,2001,3,11,237765.347,0.239112,296.092747,89.305463,304.620829,294.304126,...,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93,1.000000,1.000000
3,2001-04-01,2001,4,11,237765.347,0.334660,296.186143,88.590375,304.168669,293.921815,...,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93,1.000000,1.000000
4,2001-05-01,2001,5,11,237765.347,0.378931,295.562972,86.939606,303.903043,293.395959,...,723839,97.26,27.15,144.93,9.42,51.72,53.83,36.93,32.159859,17.186546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,2019-08-01,2019,8,53,5760.784,0.362744,282.150351,42.202163,304.210083,287.271135,...,1361053,99.91,23.48,171.62,3.66,71.62,76.39,61.00,29.022461,16.795465
6152,2019-09-01,2019,9,53,5760.784,0.317748,281.820936,34.023500,307.566780,290.719267,...,1361053,99.91,23.48,171.62,3.66,71.62,76.39,61.00,20.277210,5.658783
6153,2019-10-01,2019,10,53,5760.784,0.271795,286.196146,45.486547,307.716003,291.720099,...,1361053,99.91,23.48,171.62,3.66,71.62,76.39,61.00,7.219064,16.862005
6154,2019-11-01,2019,11,53,5760.784,0.235493,290.445969,64.916154,306.706715,291.496597,...,1361053,99.91,23.48,171.62,3.66,71.62,76.39,61.00,17.866333,28.619926


**'Clean' the dataset (e.g. remove NaN values)**

In [53]:
dataframe = utils.clean(dataframe)
dataframe.head()

,Date,Year,Month,CD_UF,area_km2,NDVI_d,dewpoint_temperature_2m_d,humidity_d,max_temperature_2m_d,min_temperature_2m_d,...,t_densidadem2,rdpc_def_vulner,t_analf_18m,t_formal_18m,t_fundc_ocup18m,t_medioc_ocup18m,CNN_all,CNN_0-19,rate_total,rate_019
0,2001-01-01,2001,1,11,237765.347,0.154301,295.674980,88.460308,303.987216,294.155015,...,27.15,144.93,9.42,51.72,53.83,36.93,1.000000,1.000000,42.754490,29.124122
1,2001-02-01,2001,2,11,237765.347,0.216873,295.944060,88.856948,304.738755,294.332566,...,27.15,144.93,9.42,51.72,53.83,36.93,1.000000,1.000000,17.601025,11.718582
2,2001-03-01,2001,3,11,237765.347,0.239112,296.092747,89.305463,304.620829,294.304126,...,27.15,144.93,9.42,51.72,53.83,36.93,1.000000,1.000000,11.072645,6.376287
3,2001-04-01,2001,4,11,237765.347,0.334660,296.186143,88.590375,304.168669,293.921815,...,27.15,144.93,9.42,51.72,53.83,36.93,1.000000,1.000000,5.120298,3.791306
4,2001-05-01,2001,5,11,237765.347,0.378931,295.562972,86.939606,303.903043,293.395959,...,27.15,144.93,9.42,51.72,53.83,36.93,32.159859,17.186546,6.976406,4.652966


In [54]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6156 entries, 0 to 6155
Data columns (total 61 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       6156 non-null   object 
 1   Year                       6156 non-null   int64  
 2   Month                      6156 non-null   int64  
 3   CD_UF                      6156 non-null   int64  
 4   area_km2                   6156 non-null   float64
 5   NDVI_d                     6156 non-null   float64
 6   dewpoint_temperature_2m_d  6156 non-null   float64
 7   humidity_d                 6156 non-null   float64
 8   max_temperature_2m_d       6156 non-null   float64
 9   min_temperature_2m_d       6156 non-null   float64
 10  surface_pressure_d         6156 non-null   float64
 11  temperature_2m_d           6156 non-null   float64
 12  total_precipitation_d      6156 non-null   float64
 13  u_component_of_wind_10m_d  6156 non-null   float

In [55]:
dataframe.to_csv('out.csv',index=False)

## Apply Data Reduction
**Data reduction is applied to three macro groups in order to reduce the number of variables on which the AI framework will be trained. The variables belonging to each group are set with the *PCAgroups* dictionary. The groups are:**
1. ***CLIMATIC VARIABLES***,
2. ***GEO VARIABLES***,
3. ***SOCIO VARIABLES***

In [56]:
print('\033[1m PCA Excluded Variables \033[0m')
utils.plist(GROUPED_VARS['EXCLUDED'])

print('\033[1m Climatic variables \033[0m')
utils.plist(GROUPED_VARS['CLIMATIC VARIABLES'])

print('\033[1m Geo variables \033[0m')
utils.plist(GROUPED_VARS['GEO VARIABLES'])

print('\033[1m Socio variables \033[0m')
utils.plist(GROUPED_VARS['SOCIO VARIABLES'])

print('\033[1m Additional variables \033[0m')
utils.plist(GROUPED_VARS['AUXILIAR'])

print('\033[1m Dengue variables \033[0m')
utils.plist(GROUPED_VARS['DENGUE'])

 PCA Excluded Variables 
----- 1 t_fundc_ocup18m
----- 2 t_medioc_ocup18m
----- 3 PopTotal_Urban_UF
----- 4 PopTotal_Rural_UF
----- 5 total_precipitation_d
----- 6 surface_pressure_d
----- 7 area_km2
----- 8 humidity_d
----- 9 temperature_2m_d
----- 10 min_temperature_2m_d
----- 11 CNN_all
----- 12 CNN_0-19
 Climatic variables 
----- 1 dewpoint_temperature_2m_d
----- 2 max_temperature_2m_d
----- 3 u_component_of_wind_10m_d
----- 4 v_component_of_wind_10m_d
 Geo variables 
----- 1 NDVI_d
----- 2 max_elevation_d
----- 3 mean_elevation_d
----- 4 min_elevation_d
----- 5 stdDev_elevation_d
----- 6 variance_elevation_d
----- 7 Forest_Cover_Percent
----- 8 Urban_Cover_Percent
 Socio variables 
----- 1 Urban_Cover_Percent
----- 2 ivs
----- 3 ivs_infraestrutura_urbana
----- 4 ivs_capital_humano
----- 5 ivs_renda_e_trabalho
----- 6 t_sem_agua_esgoto
----- 7 t_sem_lixo
----- 8 t_vulner_mais1h
----- 9 t_analf_15m
----- 10 t_cdom_fundin
----- 11 t_p15a24_nada
----- 12 t_vulner
----- 13 t_desocup18m

**We selected two types of data reduction methods: PCA (Principal Component Analysis) and PLS (Principal Least Square). The second one is the default solution because it reduces the input data by considering also a second variable that in our case is the Dengue Incidence Rates.**

In [57]:
from data_reduction import pca_reducer, pls_reducer

**Extract climatic, geophysical and socio-economic variables from the dataframe**

In [58]:
X_climatic = dataframe[GROUPED_VARS['CLIMATIC VARIABLES']].values
X_geo = dataframe[GROUPED_VARS['GEO VARIABLES']].values
X_socio = dataframe[GROUPED_VARS['SOCIO VARIABLES']].values


**Extract Dengue variables from the dataframe, apply a root scaling and normalization**

In [59]:
y_dengue = dataframe[GROUPED_VARS['DENGUE']].values
scaler = MinMaxScaler()
y_dengue = scaler.fit_transform(y_dengue)

**Apply data reduction technique**

In [60]:
if DATA_REDUCER_SETTINGS['TYPE'] == 'PLS':
    climatic_vars_reduced = pls_reducer(
        X_climatic,
        y_dengue,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['CLIMATIC VARIABLES'])

    geo_vars_reduced = pls_reducer(
        X_geo,
        y_dengue,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['GEO VARIABLES'])

    socio_vars_reduced = pls_reducer(
        X_socio,
        y_dengue,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['SOCIO VARIABLES'])
    print(socio_vars_reduced.shape    )
elif DATA_REDUCER_SETTINGS['TYPE'] == 'PCA':
    climatic_vars_reduced = pca_reducer(
        X_climatic,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['CLIMATIC VARIABLES'])

    geo_vars_reduced = pca_reducer(
        X_geo,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['GEO VARIABLES'])

    socio_vars_reduced = pca_reducer(
        X_socio,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['SOCIO VARIABLES'])
else:
    print('No data reduction.')

(6156, 10)


## Order reduced data in a new dataframe


**Normalize remaining variables**

In [61]:
x_excluded = dataframe[GROUPED_VARS['EXCLUDED']].values
x_excluded = MinMaxScaler().fit_transform(x_excluded)

In [62]:
X_auxiliar = dataframe[GROUPED_VARS['AUXILIAR']].values
X_auxiliar = MinMaxScaler().fit_transform(X_auxiliar)

**Create a new database with the reduced, the auxiliar and Dengue variables**

In [63]:
independent = {'Year':dataframe.Year.values, 'dep_id':dataframe.CD_UF.values, 't_fundc_ocup18m':x_excluded[:, 0], 't_medioc_ocup18m':x_excluded[:, 1],
               'PopTotal_Urban_UF':x_excluded[:, 2], 'PopTotal_Rural_UF':x_excluded[:, 3], 'total_precipitation_d':x_excluded[:, 4],
               'surface_pressure_d':x_excluded[:, 5], 'area_km2':x_excluded[:, 6], 'humidity_d':x_excluded[:, 7], 'temperature_2m_d':x_excluded[:, 8],
               'min_temperature_2m_d':x_excluded[:, 9], 'CNN_all':x_excluded[:, 10], 'CNN_0-19':x_excluded[:, 11]}

auxiliar    = {'Month': X_auxiliar[:, 0],
               'cases20_99': X_auxiliar[:, 1], 'cases0_19': X_auxiliar[:, 2],
               'RandEffects1':  MinMaxScaler().fit_transform(np.reshape(dataframe.CD_UF.values*dataframe.Month.values, (dataframe.CD_UF.values.shape[0], 1)))[:,0],
               'RandEffects2':  MinMaxScaler().fit_transform(np.reshape(dataframe.CD_UF.values*dataframe.Year.values, (dataframe.CD_UF.values.shape[0], 1)))[:,0],
               'RandEffects3':  MinMaxScaler().fit_transform(np.reshape(dataframe.CD_UF.values*dataframe.Month.values*dataframe.Year.values, (dataframe.CD_UF.values.shape[0], 1)))[:,0]}

climatic    = {'PCA0-Climatic':climatic_vars_reduced[:,0], 'PCA1-Climatic':climatic_vars_reduced[:,1], 'PCA2-Climatic':climatic_vars_reduced[:,2],
               'PCA3-Climatic':climatic_vars_reduced[:,3]}

geo         = {'PCA0-Geo':geo_vars_reduced[:,0], 'PCA1-Geo':geo_vars_reduced[:,1], 'PCA2-Geo':geo_vars_reduced[:,2],
               'PCA3-Geo':geo_vars_reduced[:,3], 'PCA4-Geo':geo_vars_reduced[:,4], 'PCA5-Geo':geo_vars_reduced[:,5]}

socio       = {'PCA0-Socio':socio_vars_reduced[:,0], 'PCA1-Socio':socio_vars_reduced[:,1], 'PCA2-Socio':socio_vars_reduced[:,2],
               'PCA3-Socio':socio_vars_reduced[:,3], 'PCA4-Socio':socio_vars_reduced[:,4], 'PCA5-Socio':socio_vars_reduced[:,5]}

dengue      = {'DengRate_all': y_dengue[:,0], 'DengRate_019': y_dengue[:,1]}

columns     = {**independent, **auxiliar, **climatic, **geo, **socio, **dengue}

reduced_dataframe = pd.DataFrame(columns)
reduced_dataframe.head()
reduced_dataframe.columns

# reduced_dataframe

Index(['Year', 'dep_id', 't_fundc_ocup18m', 't_medioc_ocup18m',
       'PopTotal_Urban_UF', 'PopTotal_Rural_UF', 'total_precipitation_d',
       'surface_pressure_d', 'area_km2', 'humidity_d', 'temperature_2m_d',
       'min_temperature_2m_d', 'CNN_all', 'CNN_0-19', 'Month', 'cases20_99',
       'cases0_19', 'RandEffects1', 'RandEffects2', 'RandEffects3',
       'PCA0-Climatic', 'PCA1-Climatic', 'PCA2-Climatic', 'PCA3-Climatic',
       'PCA0-Geo', 'PCA1-Geo', 'PCA2-Geo', 'PCA3-Geo', 'PCA4-Geo', 'PCA5-Geo',
       'PCA0-Socio', 'PCA1-Socio', 'PCA2-Socio', 'PCA3-Socio', 'PCA4-Socio',
       'PCA5-Socio', 'DengRate_all', 'DengRate_019'],
      dtype='object')

In [64]:
reduced_dataframe=reduced_dataframe[reduced_dataframe['Year']>=2004]
reduced_dataframe.to_csv('reduced_2004_2019.csv', index=False)
reduced_dataframe.shape

(5184, 38)

In [ ]:
# reduced_dataframe=pd.read_csv('reduced_2004_2019.csv')

## Create training and validation data
**First of all, the dataframe is divided in two sub-dataframes (training and validation) by using the variable *Year***

In [104]:
# reduced_dataframe=pd.read_csv('../google trends/merged_dataset.csv')
reduced_dataframe=pd.read_csv('../google trends/merged_dataset_lagged.csv')
print(reduced_dataframe.shape)
print(reduced_dataframe.columns)

(5172, 83)
Index(['Year', 'dep_id', 't_fundc_ocup18m', 't_medioc_ocup18m',
       'PopTotal_Urban_UF', 'PopTotal_Rural_UF', 'total_precipitation_d',
       'surface_pressure_d', 'area_km2', 'humidity_d', 'temperature_2m_d',
       'min_temperature_2m_d', 'CNN_all', 'CNN_0-19', 'Month', 'cases20_99',
       'cases0_19', 'RandEffects1', 'RandEffects2', 'RandEffects3',
       'PCA0-Climatic', 'PCA1-Climatic', 'PCA2-Climatic', 'PCA3-Climatic',
       'PCA0-Geo', 'PCA1-Geo', 'PCA2-Geo', 'PCA3-Geo', 'PCA4-Geo', 'PCA5-Geo',
       'PCA0-Socio', 'PCA1-Socio', 'PCA2-Socio', 'PCA3-Socio', 'PCA4-Socio',
       'PCA5-Socio', 'mosquito_interest', 'sintomas_dengue_interest',
       'dengue_interest', 'DengRate_all', 'DengRate_019', 'CNN_all_lag1',
       'CNN_0-19_lag1', 'humidity_d_lag2', 'humidity_d_lag5',
       'min_temperature_2m_d_lag7', 'temperature_2m_d_lag7',
       'total_precipitation_d_lag3', 'total_precipitation_d_lag5',
       'surface_pressure_d_lag6', 'PCA0-Climatic_lag5', 'PCA1-Clim

In [105]:
training_dataframe = reduced_dataframe[reduced_dataframe.Year <= 2017]
validation_dataframe = reduced_dataframe[reduced_dataframe.Year >= 2017]
print(len(training_dataframe),
len(validation_dataframe))
training_dataframe.head()
validation_dataframe.head()

4524 972


,Year,dep_id,t_fundc_ocup18m,t_medioc_ocup18m,PopTotal_Urban_UF,PopTotal_Rural_UF,total_precipitation_d,surface_pressure_d,area_km2,humidity_d,...,surface_pressure_d_roll3_lag6,PCA0-Climatic_roll3_lag5,PCA1-Climatic_roll3_lag7,PCA2-Climatic_roll3_lag6,PCA3-Climatic_roll3_lag7,humidity_d_ewma0.7_lag5,total_precipitation_d_ewma0.6_lag6,dengue_interest_ewma0.8_lag1,sintomas_dengue_interest_ewma0.8_lag2,mosquito_interest_ewma0.7_lag7
144,2017,11,0.206751,0.12853,0.020497,0.089637,0.470017,0.816201,0.149352,0.933080,...,2.525770,2.094998,1.491881,2.255944,2.101456,0.482751,0.038149,0.113181,0.000058,0.144966
145,2017,11,0.206751,0.12853,0.020497,0.089637,0.455807,0.813839,0.149352,0.957511,...,2.514451,2.145001,1.334171,2.457732,2.310252,0.677117,0.058972,0.134636,0.112012,0.099490
146,2017,11,0.206751,0.12853,0.020497,0.089637,0.409449,0.815870,0.149352,0.945000,...,2.495180,2.424528,1.075503,2.467724,2.545289,0.801480,0.137642,0.138927,0.214402,0.099847
147,2017,11,0.206751,0.12853,0.020497,0.089637,0.239287,0.822995,0.149352,0.947705,...,2.468857,2.643550,0.995015,2.227274,2.696011,0.879421,0.233861,0.179785,0.322880,0.106954
148,2017,11,0.206751,0.12853,0.020497,0.089637,0.123361,0.827288,0.149352,0.935646,...,2.442081,2.772550,1.078932,1.970719,2.633818,0.935384,0.276210,0.179957,0.240576,0.081086


**Then the dataset handler is initialized. This object will handle all the operations needed to create, reshape and augment the training and validation dataset to fit the requirements of each Deep Learning or Machine Learning model.**

In [106]:
from datasetHandler import datasetHandler
dataset_handler = datasetHandler(training_dataframe, validation_dataframe)

**Get training and validation vectors from dataframes.**

In [107]:
# x_train, y_train, x_val, y_val = dataset_handler.get_data(DATA_PROCESSING_SETTINGS['T LEARNING'], DATA_PROCESSING_SETTINGS['T PREDICTION'])

x_train, y_train, x_val, y_val, train_indices, val_indices = dataset_handler.get_data(
    DATA_PROCESSING_SETTINGS['T LEARNING'], DATA_PROCESSING_SETTINGS['T PREDICTION']
)

print('\n\nX Training shape', x_train.shape)
print('Y Training shape', y_train.shape)
print('X Validation shape', x_val.shape)
print('Y Validation shape', y_val.shape)


X Training shape (3888, 12, 83)
Y Training shape (3888, 2)
Processing departments 26 of 27		
X Validation shape (648, 12, 83)
Y Validation shape (648, 2)
Processing departments 26 of 27		

X Training shape (3888, 12, 83)
Y Training shape (3888, 2)
X Validation shape (648, 12, 83)
Y Validation shape (648, 2)


**Apply data augmention**

In [108]:
def augment(self, x_train, y_train, x_val, y_val, augmentation_factor, static_train=None, static_val=None):
    # Augment x_train and y_train as usual
    x_train_augmented = np.tile(x_train, (augmentation_factor, 1, 1))
    y_train_augmented = np.tile(y_train, (augmentation_factor, 1))
    x_val_augmented = np.tile(x_val, (augmentation_factor, 1, 1))
    y_val_augmented = np.tile(y_val, (augmentation_factor, 1))
    
    # Augment static data if provided
    if static_train is not None:
        static_train_augmented = np.tile(static_train, (augmentation_factor, 1))
    else:
        static_train_augmented = None

    if static_val is not None:
        static_val_augmented = np.tile(static_val, (augmentation_factor, 1))
    else:
        static_val_augmented = None

    return x_train_augmented, y_train_augmented, x_val_augmented, y_val_augmented, static_train_augmented, static_val_augmented


In [109]:
x_train_a, y_train_a, x_val_a, y_val_a = dataset_handler.augment(x_train, y_train, x_val, y_val, DATA_PROCESSING_SETTINGS['AUGMENTATION'])
print('X Training shape', x_train_a.shape)
print('Y Training shape', y_train_a.shape)
print('X Validation shape', x_val_a.shape)
print('Y Validation shape', y_val_a.shape)

# # Augment static data to match temporal data
# static_train_a = np.repeat(static_train, repeats=DATA_PROCESSING_SETTINGS['AUGMENTATION'], axis=0)
# static_val_a = np.repeat(static_val, repeats=DATA_PROCESSING_SETTINGS['AUGMENTATION'], axis=0)

# # Validate shapes
# # Validate shapes
# print(f"x_train_a shape: {x_train_a.shape}, static_train_a shape: {static_train_a.shape}")
# print(f"x_val_a shape: {x_val_a.shape}, static_val_a shape: {static_val_a.shape}")
# assert x_train_a.shape[0] == static_train_a.shape[0], "Mismatch after augmentation: x_train_a and static_train_a"
# assert x_val_a.shape[0] == static_val_a.shape[0], "Mismatch after augmentation: x_val_a and static_val_a"

# x_train_a, y_train_a, x_val_a, y_val_a, static_train_a, static_val_a = dataset_handler.augment(
#     x_train, y_train, x_val, y_val, 
#     DATA_PROCESSING_SETTINGS['AUGMENTATION'], 
#     static_train=static_train, 
#     static_val=static_val
# )

# # Validate shapes
# print(f"x_train_a shape: {x_train_a.shape}, static_train_a shape: {static_train_a.shape}")
# print(f"x_val_a shape: {x_val_a.shape}, static_val_a shape: {static_val_a.shape}")
# assert x_train_a.shape[0] == static_train_a.shape[0], "Mismatch after augmentation: x_train_a and static_train_a"
# assert x_val_a.shape[0] == static_val_a.shape[0], "Mismatch after augmentation: x_val_a and static_val_a"

X Training shape (11664, 12, 83)
Y Training shape (11664, 2)
X Validation shape (1944, 12, 83)
Y Validation shape (1944, 2)


# TCN

In [72]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, BatchNormalization, Dropout, Input, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics import root_mean_squared_error
from keras.metrics import MeanSquaredError, MeanAbsoluteError
from keras_tuner import HyperModel, RandomSearch
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from datetime import datetime
from glob import glob
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error


In [73]:
from tensorflow.keras import layers, models, regularizers, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

LSTM_SETTINGS = {
    'EPOCHS': 200,
    'LEARNING RATE': 0.0001,
    'BATCH SIZE': 16,
    'OPTIMZER': 'rmsprop', #'adam',
    'LOSS':'mae',
    'EVALUATION METRIC':['mse'],
    'EARLY STOPPING': 12
}

def build_tcn_model_v2(input_shape, output_units):
    def residual_block(x, filters, dilation_rate):
        shortcut = x  # Residual connection
        x = layers.Conv1D(filters, kernel_size=3, padding='causal', 
                          activation='relu', dilation_rate=dilation_rate,
                          kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        x = layers.Conv1D(filters, kernel_size=3, padding='causal', 
                          activation='relu', dilation_rate=dilation_rate,
                          kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        # Add residual connection
        x = layers.add([x, shortcut])
        x = layers.ReLU()(x)
        return x

    inputs = Input(shape=input_shape)
    x = layers.Conv1D(filters=64, kernel_size=3, padding='causal', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    
    # Add residual blocks with increasing dilation rates
    for dilation_rate in [1, 2, 4, 8]:
        x = residual_block(x, filters=64, dilation_rate=dilation_rate)
    
    x = layers.Flatten()(x)
    outputs = layers.Dense(units=output_units, activation='linear')(x)  # For regression
    
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='Huber', metrics=['mae'])
    return model

class ImprovedTCNNet:
    def __init__(self, shape, output_units=2):
        self.shape = shape
        self.epochs = LSTM_SETTINGS['EPOCHS']  # Max epochs
        self.batch_size = LSTM_SETTINGS['BATCH SIZE']
        self.lr = LSTM_SETTINGS['LEARNING RATE']
        self.early_stopping_rounds = LSTM_SETTINGS['EARLY STOPPING']
        
        self.model = build_tcn_model_v2(self.shape, output_units)
    
    def load(self, model_path):
            """Load a saved model from the specified path."""
            self.model = tf.keras.models.load_model(model_path)
            print(f"Model loaded successfully from {model_path}")
        
    def train(self, training, validation, output_path):
        # Early stopping and learning rate scheduler
        es = EarlyStopping(
            monitor='val_loss',
            patience=self.early_stopping_rounds,
            restore_best_weights=True
        )
        
        lr_scheduler = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
        
        # Train the model
        history = self.model.fit(
            x=training[0],
            y=training[1],
            validation_data=(validation[0], validation[1]),
            epochs=self.epochs,
            batch_size=self.batch_size,
            callbacks=[es, lr_scheduler],
            shuffle=True
        )

        # Plot the training history
        plt.figure(figsize=(8, 6))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.legend()
        plt.title('Improved Model Loss Curve')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.show()

        # Save the model
        today = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
        self.model.save(os.path.join(output_path, 'TCN-new-lagged-' + today + '.keras'))
        return history

# Use the improved model
trainingT, validationT = dataset_handler.prepare_data_LSTM(x_train_a[:,:,2:], y_train_a, x_val_a[:,:,2:], y_val_a)
tcn = ImprovedTCNNet(trainingT[0].shape[1:])



In [74]:
# Define paths
output_path = os.path.join(config['output'], "Brazil")
os.makedirs(output_path, exist_ok=True)

# Define training flag
TRAINING = False  # Set to False to skip training and load the model if it exists


# Check if training is required
if TRAINING:
    print("Training the model...")

    # history = tcn.train(trainingT, validationT, output_path=join(config['output'], "Brazil"))
    # Train the model with augmented data
    history = tcn.train(
        training=trainingT,
        validation=validationT,
        output_path=join(config['output'], "Brazil")
    )


else:
    # Check if a saved model exists
    print("Checking for saved models...")
    tcn_models = glob(os.path.join(output_path, "TCN-new-lagged-*.keras"))
    if not tcn_models:
        print('No file with such pattern was found in the directory. Run TRAINING = True first.')
        exit()
    else:
        tcn.load(tcn_models[-1])
        print(f"Loading model from: {tcn_models[-1]}")
    
    trainT, valT = dataset_handler.prepare_data_LSTM(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)
    
    # Create index mappings
    y_val_indices_df = pd.DataFrame(val_indices, columns=['actual_index'])
    y_train_indices_df = pd.DataFrame(train_indices, columns=['actual_index'])

    # Make predictions on training and validation sets
    preds_tra = tcn.model.predict(trainT[0])
    preds_tra[preds_tra < 0] = 0
    preds_val = tcn.model.predict(valT[0])
    preds_val[preds_val < 0] = 0

    results = []

    preds_val_original = scaler.inverse_transform(preds_val)
    y_val_original = scaler.inverse_transform(valT[1])
    
    # Inverse transform predictions and ground truth values for training data
    preds_tra_original = scaler.inverse_transform(preds_tra)
    y_train_original = scaler.inverse_transform(trainT[1])

    for department_idx, department_name in DEP_NAMES.items():
        # Filter rows corresponding to the current department for both training and validation
        department_rows_val = validation_dataframe[validation_dataframe['dep_id'] == department_idx]
        department_rows_train = training_dataframe[training_dataframe['dep_id'] == department_idx]
    
        if department_rows_val.empty or department_rows_train.empty:
            continue
    
        department_indices_val = department_rows_val.index.tolist()
        department_indices_train = department_rows_train.index.tolist()

        # Matching indices for validation data
        matching_indices_val = y_val_indices_df[y_val_indices_df['actual_index'].isin(department_indices_val)].index
        # Matching indices for training data
        matching_indices_train = y_train_indices_df[y_train_indices_df['actual_index'].isin(department_indices_train)].index
        
        if matching_indices_val.empty or matching_indices_train.empty:
            continue

        # Extract original scale true and predicted values for validation data
        true_dengrate_all_val = y_val_original[matching_indices_val, 0]
        true_dengrate_019_val = y_val_original[matching_indices_val, 1]
        predicted_dengrate_all_val = preds_val_original[matching_indices_val, 0]
        predicted_dengrate_019_val = preds_val_original[matching_indices_val, 1]
    
        # Extract original scale true and predicted values for training data
        true_dengrate_all_train = y_train_original[matching_indices_train, 0]
        true_dengrate_019_train = y_train_original[matching_indices_train, 1]
        predicted_dengrate_all_train = preds_tra_original[matching_indices_train, 0]
        predicted_dengrate_019_train = preds_tra_original[matching_indices_train, 1]
    
        # Calculate NRMSE for both DengRate_all and DengRate_019 (validation data)
        nrmse_dengrate_all_val = root_mean_squared_error(true_dengrate_all_val, predicted_dengrate_all_val) / (true_dengrate_all_val.max() - true_dengrate_all_val.min())
        nrmse_dengrate_019_val = root_mean_squared_error(true_dengrate_019_val, predicted_dengrate_019_val) / (true_dengrate_019_val.max() - true_dengrate_019_val.min())
    
        # Calculate NRMSE for both DengRate_all and DengRate_019 (training data)
        nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
        nrmse_dengrate_019_train = root_mean_squared_error(true_dengrate_019_train, predicted_dengrate_019_train) / (true_dengrate_019_train.max() - true_dengrate_019_train.min())
    
        # Calculate MAE for both DengRate_all and DengRate_019 (validation data)
        mae_dengrate_all_val = mean_absolute_error(true_dengrate_all_val, predicted_dengrate_all_val)
        mae_dengrate_019_val = mean_absolute_error(true_dengrate_019_val, predicted_dengrate_019_val)
    
        # Store the results
        results.append({
            'Department': department_name,
            'NRMSE 0-19 Training': nrmse_dengrate_019_train,
            'NRMSE All Training': nrmse_dengrate_all_train,
            'NRMSE 0-19 Validation': nrmse_dengrate_019_val,
            'NRMSE All Validation': nrmse_dengrate_all_val,
            'MAE (DengRate_all) Val': mae_dengrate_all_val,
            'MAE (DengRate_019) Val': mae_dengrate_019_val
        })


    # print(results)
    results_df = pd.DataFrame(results)

    #Save results
    today = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    output_path = os.path.join(config['metrics'], "Brazil", f'TCN_model_search_lagged_{today}.csv')

    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    results_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

Checking for saved models...
Model loaded successfully from e:\minor\proj\code\saved_models\Brazil\TCN-new-lagged-18-03-2025-12-09-44.keras
Loading model from: e:\minor\proj\code\saved_models\Brazil\TCN-new-lagged-18-03-2025-12-09-44.keras
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Results saved to e:\minor\proj\code\metrics\Brazil\TCN_model_search_lagged_18-03-2025-21-59-40.csv


C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\2698071502.py:90: RuntimeWarning: divide by zero encountered in scalar divide
  nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\2698071502.py:90: RuntimeWarning: divide by zero encountered in scalar divide
  nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\2698071502.py:87: RuntimeWarning: divide by zero encountered in scalar divide
  nrmse_dengrate_019_val = root_mean_squared_error(true_dengrate_019_val, predicted_dengrate_019_val) / (true_dengrate_019_val.max() - true_dengrate_019_val.min())


# LSTM

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras.metrics import MeanSquaredError, MeanAbsoluteError
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import matplotlib.pyplot as plt

In [76]:
trainingL, validationL = dataset_handler.prepare_data_LSTM(x_train_a[:,:,2:], y_train_a, x_val_a[:,:,2:], y_val_a)

In [33]:
LSTM_SETTINGS = {
    'EPOCHS': 200,
    'LEARNING RATE': 0.0001,
    'BATCH SIZE': 16,
    'OPTIMZER': 'rmsprop', #'adam',
    'LOSS':'mae',
    'EVALUATION METRIC':['mse'],
    'EARLY STOPPING': 12
}

def custom_load_model(path):
    return load_model(
        path,
        custom_objects={
            'mae': MeanAbsoluteError(),
            'mse': MeanSquaredError(),
        },
        compile=True
    )
    
class LSTMNet:
    def __init__(self, shape):
        self.shape = shape
        self.epochs = LSTM_SETTINGS['EPOCHS']
        self.lr = LSTM_SETTINGS['LEARNING RATE']
        self.batch_size = LSTM_SETTINGS['BATCH SIZE']
        self.loss = LSTM_SETTINGS['LOSS']
        self.eval_metric = LSTM_SETTINGS['EVALUATION METRIC']
        self.early_stopping_rounds = LSTM_SETTINGS['EARLY STOPPING']
        
        # Use .get() to avoid KeyError if 'OPTIMIZER' is missing
        self.optimizer = LSTM_SETTINGS.get('OPTIMIZER', 'adam')
        if self.optimizer == 'adam':
            self.optimizer = Adam(learning_rate=self.lr)
        elif self.optimizer == 'rmsprop':
            self.optimizer = RMSprop(learning_rate=self.lr)
        else:
            self.optimizer = Adam(learning_rate=self.lr)

        self.model = self.__build()

    def __build(self):
        model = Sequential()
        model.add(Input(shape=self.shape))
        model.add(LSTM(60, return_sequences=True, dropout=0.5))
        model.add(LSTM(20, dropout=0.5))
        model.add(Dense(2))
        model.compile(loss=self.loss, metrics=self.eval_metric, optimizer=self.optimizer)
        return model

    # def load(self, path):
    #     self.model = load_model(path)
    def load(self, path):
        self.model = custom_load_model(path)

    def train(self, training, validation, output_path):
        es = EarlyStopping(
            monitor='val_loss', 
            min_delta=0, 
            patience=self.early_stopping_rounds, 
            verbose=0, 
            mode='auto', 
            baseline=None, 
            restore_best_weights=True
        )

        history = self.model.fit(
            x=training[0],
            y=training[1],
            validation_data=(validation[0], validation[1]),
            epochs=self.epochs,
            batch_size=self.batch_size,
            callbacks=[es],
            shuffle=True
        )

            # Plot the training history
        plt.figure(figsize=(8, 6))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.legend()
        plt.title('Model Loss Curve')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.show()

        today = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
        self.model.save(os.path.join(output_path, 'LSTM-new-lagged-' + today + '.h5'))
        return history

lstm = LSTMNet(trainingL[0].shape[1:])


In [39]:
TRAINING = False

if TRAINING:
    lstm.train(trainingL, validationL, output_path=join(config['output'], "Brazil"))

else:
    # get most recent lstm model
    lstm_model = glob(join(config['output'], "Brazil", "LSTM-new-lagged-*"))
    if not lstm_model:
        print('No file with such pattern was found in the directory. Run TRAINING = True first.')
        exit()
    else:
        lstm.load(lstm_model[-1])
        print(f"Loading model from: {lstm_model[-1]}")

    trainL, valL = dataset_handler.prepare_data_LSTM(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)
    
    preds_tra = lstm.model.predict(trainL[0])
    preds_tra[preds_tra < 0] = 0
    preds_val = lstm.model.predict(valL[0])
    preds_val[preds_val < 0] = 0
    # print("preds_tra: \n",preds_tra.shape,"\n")
    # print("preds_val: \n",preds_val.shape,"\n")
    
        # Align predictions with department indices
    # y_val_indices = validation_dataframe.index.to_list()
    # y_val_indices_df = pd.DataFrame(y_val_indices, columns=['actual_index'])
    
    # y_train_indices = training_dataframe.index.to_list()
    # y_train_indices_df = pd.DataFrame(y_train_indices, columns=['actual_index'])

    # Create index mappings
    y_val_indices_df = pd.DataFrame(val_indices, columns=['actual_index'])
    y_train_indices_df = pd.DataFrame(train_indices, columns=['actual_index'])
    
    # Initialize results list
    results = []
    preds_val_original = scaler.inverse_transform(preds_val)
    y_val_original = scaler.inverse_transform(valL[1])
    
    # Inverse transform predictions and ground truth values for training data
    preds_tra_original = scaler.inverse_transform(preds_tra)
    y_train_original = scaler.inverse_transform(trainL[1])
    
    for department_idx, department_name in DEP_NAMES.items():
        # Filter rows corresponding to the current department for both training and validation
        department_rows_val = validation_dataframe[validation_dataframe['dep_id'] == department_idx]
        department_rows_train = training_dataframe[training_dataframe['dep_id'] == department_idx]
    
        if department_rows_val.empty or department_rows_train.empty:
            continue
    
        department_indices_val = department_rows_val.index.tolist()
        department_indices_train = department_rows_train.index.tolist()
    
        # # Matching indices for validation data
        # matching_indices_val = y_val_indices_df[y_val_indices_df['actual_index'].isin(department_indices_val)].index
        # matching_indices_train = y_train_indices_df[y_train_indices_df['actual_index'].isin(department_indices_train)].index

        # Matching indices for validation data
        matching_indices_val = y_val_indices_df[y_val_indices_df['actual_index'].isin(department_indices_val)].index
        # Matching indices for training data
        matching_indices_train = y_train_indices_df[y_train_indices_df['actual_index'].isin(department_indices_train)].index
        
        if matching_indices_val.empty or matching_indices_train.empty:
            continue
    
        # Extract original scale true and predicted values for validation data
        true_dengrate_all_val = y_val_original[matching_indices_val, 0]
        true_dengrate_019_val = y_val_original[matching_indices_val, 1]
        predicted_dengrate_all_val = preds_val_original[matching_indices_val, 0]
        predicted_dengrate_019_val = preds_val_original[matching_indices_val, 1]
    
        # Extract original scale true and predicted values for training data
        true_dengrate_all_train = y_train_original[matching_indices_train, 0]
        true_dengrate_019_train = y_train_original[matching_indices_train, 1]
        predicted_dengrate_all_train = preds_tra_original[matching_indices_train, 0]
        predicted_dengrate_019_train = preds_tra_original[matching_indices_train, 1]
    
        # Calculate NRMSE for both DengRate_all and DengRate_019 (validation data)
        nrmse_dengrate_all_val = root_mean_squared_error(true_dengrate_all_val, predicted_dengrate_all_val) / (true_dengrate_all_val.max() - true_dengrate_all_val.min())
        nrmse_dengrate_019_val = root_mean_squared_error(true_dengrate_019_val, predicted_dengrate_019_val) / (true_dengrate_019_val.max() - true_dengrate_019_val.min())
    
        # Calculate NRMSE for both DengRate_all and DengRate_019 (training data)
        nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
        nrmse_dengrate_019_train = root_mean_squared_error(true_dengrate_019_train, predicted_dengrate_019_train) / (true_dengrate_019_train.max() - true_dengrate_019_train.min())
    
        # Calculate MAE for both DengRate_all and DengRate_019 (validation data)
        mae_dengrate_all_val = mean_absolute_error(true_dengrate_all_val, predicted_dengrate_all_val)
        mae_dengrate_019_val = mean_absolute_error(true_dengrate_019_val, predicted_dengrate_019_val)
    
        # Store the results
        results.append({
            'Department': department_name,
            'NRMSE 0-19 Training': nrmse_dengrate_019_train,
            'NRMSE All Training': nrmse_dengrate_all_train,
            'NRMSE 0-19 Validation': nrmse_dengrate_019_val,
            'NRMSE All Validation': nrmse_dengrate_all_val,
            'MAE (DengRate_all) Val': mae_dengrate_all_val,
            'MAE (DengRate_019) Val': mae_dengrate_019_val
        })

    # print(results)
    results_df = pd.DataFrame(results)

    #Save results
    today = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    output_path = os.path.join(config['metrics'], "Brazil", f'LSTM_Model_search_lagged_{today}.csv')

    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    results_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")


Loading model from: e:\minor\proj\code\saved_models\Brazil\LSTM-new-lagged-18-03-2025-20-11-52.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Results saved to e:\minor\proj\code\metrics\Brazil\LSTM_Model_search_lagged_18-03-2025-20-16-03.csv


C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\1887462693.py:85: RuntimeWarning: divide by zero encountered in scalar divide
  nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\1887462693.py:85: RuntimeWarning: divide by zero encountered in scalar divide
  nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\1887462693.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  nrmse_dengrate_019_val = root_mean_squared_error(true_dengrate_019_val, predicted_dengrate_019_val) / (true_dengrate_019_val.max() - true_dengrate_019_val.min())


# CatBoost
**Prepare dataset to fit CatBoost requirements**

In [130]:
from keras.metrics import MeanSquaredError, MeanAbsoluteError
from sklearn.metrics import mean_absolute_error

In [131]:
trainingC, validationC = dataset_handler.prepare_data_CatBoost(x_train_a[:,:,2:], y_train_a, x_val_a[:,:,2:], y_val_a)

**Initilize the CatBoost**

In [132]:
from models import CatBoostNet
cat_boost = CatBoostNet()

In [133]:
cat_boost.model.get_params()

{'iterations': 8000,
 'learning_rate': 0.01,
 'loss_function': 'MultiRMSE',
 'random_seed': 42,
 'verbose': True,
 'eval_metric': 'MultiRMSE',
 'task_type': 'CPU',
 'max_depth': 7,
 'early_stopping_rounds': 200}

**Train or Test**

In [134]:
def compute_scores(gt, preds):
    # Compute Normalized-RMSE and R2 metrics
    rmse = root_mean_squared_error(gt, preds)
    nrmse = rmse/(gt.max() - gt.min())
    r2 = r2_score(gt, preds)
    return nrmse, r2

In [137]:
TRAINING = False
# nb_deps = 27

if TRAINING:
    # cat_boost.train(trainingC, validationC, output_path=join(config['output'], "Brazil"))

    snapshot_path = os.path.join(config['output'], "Brazil", f"snapshot-{today}.bkp")
    cat_boost.train(
        trainingC,
        validationC,
        output_path=os.path.join(config['output'], "Brazil", f"CATBOOST-lagged-{today}"),
        snapshot_file=snapshot_path,
        snapshot_interval=300  # Use a new snapshot
    )

else:
    # get most recent catboost model
    cat_boost_model = glob(join(config['output'], "Brazil", "CATBOOST-lagged-*"))
    if not cat_boost_model:
        print('No file with such pattern was found in the directory. Run TRAINING = True first.')
        exit()
    else:
        cat_boost.load(cat_boost_model[-1])
        print(f"Loading model from: {cat_boost_model[-1]}")

    trainC, valC = dataset_handler.prepare_data_CatBoost(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)
    
    preds_tra = cat_boost.model.predict(trainC[0])
    preds_tra[preds_tra < 0] = 0
    preds_val = cat_boost.model.predict(valC[0])
    preds_val[preds_val < 0] = 0
    # print("preds_tra: \n",preds_tra.shape,"\n")
    # print("preds_val: \n",preds_val.shape,"\n")
    
        # Align predictions with department indices
    # y_val_indices = validation_dataframe.index.to_list()
    # y_val_indices_df = pd.DataFrame(y_val_indices, columns=['actual_index'])
    
    # y_train_indices = training_dataframe.index.to_list()
    # y_train_indices_df = pd.DataFrame(y_train_indices, columns=['actual_index'])

    # Create index mappings
    y_val_indices_df = pd.DataFrame(val_indices, columns=['actual_index'])
    y_train_indices_df = pd.DataFrame(train_indices, columns=['actual_index'])

    # Initialize results list
    results = []
    preds_val_original = scaler.inverse_transform(preds_val)
    y_val_original = scaler.inverse_transform(valL[1])
    
    # Inverse transform predictions and ground truth values for training data
    preds_tra_original = scaler.inverse_transform(preds_tra)
    y_train_original = scaler.inverse_transform(trainL[1])
    
    for department_idx, department_name in DEP_NAMES.items():
        # Filter rows corresponding to the current department for both training and validation
        department_rows_val = validation_dataframe[validation_dataframe['dep_id'] == department_idx]
        department_rows_train = training_dataframe[training_dataframe['dep_id'] == department_idx]
    
        if department_rows_val.empty or department_rows_train.empty:
            continue
    
        department_indices_val = department_rows_val.index.tolist()
        department_indices_train = department_rows_train.index.tolist()
    
        # Matching indices for validation data
        # matching_indices_val = y_val_indices_df[y_val_indices_df['actual_index'].isin(department_indices_val)].index
        # matching_indices_train = y_train_indices_df[y_train_indices_df['actual_index'].isin(department_indices_train)].index

        # Matching indices for validation data
        matching_indices_val = y_val_indices_df[y_val_indices_df['actual_index'].isin(department_indices_val)].index
        # Matching indices for training data
        matching_indices_train = y_train_indices_df[y_train_indices_df['actual_index'].isin(department_indices_train)].index
    
        if matching_indices_val.empty or matching_indices_train.empty:
            continue
    
        # Extract original scale true and predicted values for validation data
        true_dengrate_all_val = y_val_original[matching_indices_val, 0]
        true_dengrate_019_val = y_val_original[matching_indices_val, 1]
        predicted_dengrate_all_val = preds_val_original[matching_indices_val, 0]
        predicted_dengrate_019_val = preds_val_original[matching_indices_val, 1]
    
        # Extract original scale true and predicted values for training data
        true_dengrate_all_train = y_train_original[matching_indices_train, 0]
        true_dengrate_019_train = y_train_original[matching_indices_train, 1]
        predicted_dengrate_all_train = preds_tra_original[matching_indices_train, 0]
        predicted_dengrate_019_train = preds_tra_original[matching_indices_train, 1]
    
        # Calculate NRMSE for both DengRate_all and DengRate_019 (validation data)
        nrmse_dengrate_all_val = root_mean_squared_error(true_dengrate_all_val, predicted_dengrate_all_val) / (true_dengrate_all_val.max() - true_dengrate_all_val.min())
        nrmse_dengrate_019_val = root_mean_squared_error(true_dengrate_019_val, predicted_dengrate_019_val) / (true_dengrate_019_val.max() - true_dengrate_019_val.min())
    
        # Calculate NRMSE for both DengRate_all and DengRate_019 (training data)
        nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
        nrmse_dengrate_019_train = root_mean_squared_error(true_dengrate_019_train, predicted_dengrate_019_train) / (true_dengrate_019_train.max() - true_dengrate_019_train.min())
    
        # Calculate MAE for both DengRate_all and DengRate_019 (validation data)
        mae_dengrate_all_val = mean_absolute_error(true_dengrate_all_val, predicted_dengrate_all_val)
        mae_dengrate_019_val = mean_absolute_error(true_dengrate_019_val, predicted_dengrate_019_val)
    
        # Store the results
        results.append({
            'Department': department_name,
            'NRMSE 0-19 Training': nrmse_dengrate_019_train,
            'NRMSE All Training': nrmse_dengrate_all_train,
            'NRMSE 0-19 Validation': nrmse_dengrate_019_val,
            'NRMSE All Validation': nrmse_dengrate_all_val,
            'MAE (DengRate_all) Val': mae_dengrate_all_val,
            'MAE (DengRate_019) Val': mae_dengrate_019_val
        })


    # print(results)
    results_df = pd.DataFrame(results)

    #Save results
    today = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    output_path = os.path.join(config['metrics'], "Brazil", f'catboost_lagged_{today}.csv')

    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    results_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")


Loading model from: e:\minor\proj\code\saved_models\Brazil\CATBOOST-lagged-18-03-2025-23-03-37
Results saved to e:\minor\proj\code\metrics\Brazil\catboost_lagged_19-03-2025-00-47-00.csv


C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\3864581728.py:95: RuntimeWarning: invalid value encountered in scalar divide
  nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\3864581728.py:95: RuntimeWarning: divide by zero encountered in scalar divide
  nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\3864581728.py:92: RuntimeWarning: divide by zero encountered in scalar divide
  nrmse_dengrate_019_val = root_mean_squared_error(true_dengrate_019_val, predicted_dengrate_019_val) / (true_dengrate_019_val.max() - true_dengrate_019_val.min())


# TFT

In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os
import pandas as pd
from sklearn.metrics import mean_squared_error

# Dataset Class
class TFTDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.tensor(x_data, dtype=torch.float32)
        self.y_data = torch.tensor(y_data, dtype=torch.float32)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

# Temporal Fusion Transformer Model
class TemporalFusionTransformer(nn.Module):
    def __init__(self, input_dim, output_dim=2, hidden_size=128, num_heads=8):
        super(TemporalFusionTransformer, self).__init__()
        self.hidden_size = hidden_size

        # LSTM for time-varying input (reduced dominance)
        self.lstm = nn.LSTM(input_dim, hidden_size, batch_first=True, dropout=0.4, num_layers=1)

        # Attention layer (more emphasis)
        self.attention = nn.MultiheadAttention(hidden_size, num_heads, batch_first=True, dropout=0.2)
        self.layer_norm = nn.LayerNorm(hidden_size)

        # Final output layers
        self.output_dense = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, output_dim)
        )

    def forward(self, x):
        # LSTM processing
        lstm_out, _ = self.lstm(x)

        # Attention layer
        attn_output, _ = self.attention(lstm_out, lstm_out, lstm_out)
        attn_output = self.layer_norm(attn_output + lstm_out)  # Residual connection

        # Final output layer
        output = self.output_dense(attn_output[:, -1, :])  # Use the last time step
        return output

# Training Loop
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

def train_model(model, train_loader, val_loader, optimizer, criterion, device, num_epochs):
    history = {'train_loss': [], 'val_loss': []}
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)
    early_stopping = EarlyStopping(patience=10, min_delta=0.001)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * x_batch.size(0)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item() * x_batch.size(0)

        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        scheduler.step(val_loss)
        if early_stopping(val_loss):
            print("Early stopping triggered.")
            break
    
    return history

# Main Execution Pipeline
def main_pipeline(x_train, y_train, x_val, y_val, model_path, num_epochs=80):
    input_dim = x_train.shape[2]
    output_dim = y_train.shape[1]
    batch_size = 32
    
    train_dataset = TFTDataset(x_train, y_train)
    val_dataset = TFTDataset(x_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = TemporalFusionTransformer(input_dim, output_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    history = train_model(model, train_loader, val_loader, optimizer, criterion, device, num_epochs)
    
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

    today = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    model_save_path = os.path.join(model_path, 'TFT_new_model_' + today + '.pt')
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")
    return model, model_save_path


In [103]:
import os
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define paths
output_path = os.path.join(config['output'], "Brazil")
os.makedirs(output_path, exist_ok=True)

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

TRAINING=False

# Dynamically determine input_dim and output_dim based on non-augmented data for consistency
input_dim = x_train[:, :, 2:].shape[2]  # Sliced feature size
output_dim = y_train.shape[1]          # Output size (targets)


# Train the model using augmented data
if TRAINING:
    print("Training the model with augmented data...")
    model, model_path = main_pipeline(
        x_train_a[:, :, 2:], y_train_a,  # Augmented data for training
        x_val_a[:, :, 2:], y_val_a,  # Augmented validation data
        # static_train_a, static_val_a,
        model_path=output_path,
        num_epochs=80
    )
    print(f"Model saved to {model_path}")
else:
    print("Checking for saved models...")
    tft_models = glob(os.path.join(output_path, "TFT_new_model_*.pt"))
    if not tft_models:
        print('No file with such pattern was found in the directory. Run TRAINING = True first.')
        raise FileNotFoundError("Saved model not found. Train the model first.")
    else:
        # Load the latest saved model
        model_path = tft_models[-1]
        print(f"Loading model from: {model_path}")
        model = TemporalFusionTransformer(input_dim, output_dim, hidden_size=128, num_heads=8)
        model.load_state_dict(torch.load(model_path))
        model.eval()
    
    trainT, valT = dataset_handler.prepare_data_LSTM(x_train[:, :, 2:], y_train, x_val[:, :, 2:], y_val)

    # Ensure the model dimensions match the data
    assert trainT[0].shape[-1] == input_dim, f"Mismatch: input_dim={input_dim}, trainT features={trainT[0].shape[-1]}"

    # Create index mappings
    y_val_indices_df = pd.DataFrame(val_indices, columns=['actual_index'])
    y_train_indices_df = pd.DataFrame(train_indices, columns=['actual_index'])
    
    # Make predictions on training and validation sets
    with torch.no_grad():
        preds_tra = model(torch.tensor(trainT[0], dtype=torch.float32)).numpy()
        preds_tra[preds_tra < 0] = 0
        preds_val = model(torch.tensor(valT[0], dtype=torch.float32)).numpy()
        preds_val[preds_val < 0] = 0
    
    results = []
    preds_val_original = scaler.inverse_transform(preds_val)
    y_val_original = scaler.inverse_transform(valT[1])
    
    # Inverse transform predictions and ground truth values for training data
    preds_tra_original = scaler.inverse_transform(preds_tra)
    y_train_original = scaler.inverse_transform(trainT[1])
    
    for department_idx, department_name in DEP_NAMES.items():
        # Filter rows corresponding to the current department for both training and validation
        department_rows_val = validation_dataframe[validation_dataframe['dep_id'] == department_idx]
        department_rows_train = training_dataframe[training_dataframe['dep_id'] == department_idx]
    
        if department_rows_val.empty or department_rows_train.empty:
            continue
    
        department_indices_val = department_rows_val.index.tolist()
        department_indices_train = department_rows_train.index.tolist()
    
        # Matching indices for validation data
        matching_indices_val = y_val_indices_df[y_val_indices_df['actual_index'].isin(department_indices_val)].index
        # Matching indices for training data
        matching_indices_train = y_train_indices_df[y_train_indices_df['actual_index'].isin(department_indices_train)].index
    
        if matching_indices_val.empty or matching_indices_train.empty:
            continue
    
        # Extract original scale true and predicted values for validation data
        true_dengrate_all_val = y_val_original[matching_indices_val, 0]
        true_dengrate_019_val = y_val_original[matching_indices_val, 1]
        predicted_dengrate_all_val = preds_val_original[matching_indices_val, 0]
        predicted_dengrate_019_val = preds_val_original[matching_indices_val, 1]
    
        # Extract original scale true and predicted values for training data
        true_dengrate_all_train = y_train_original[matching_indices_train, 0]
        true_dengrate_019_train = y_train_original[matching_indices_train, 1]
        predicted_dengrate_all_train = preds_tra_original[matching_indices_train, 0]
        predicted_dengrate_019_train = preds_tra_original[matching_indices_train, 1]
    
        # Calculate NRMSE for both DengRate_all and DengRate_019 (validation data)
        nrmse_dengrate_all_val = root_mean_squared_error(true_dengrate_all_val, predicted_dengrate_all_val) / (true_dengrate_all_val.max() - true_dengrate_all_val.min())
        nrmse_dengrate_019_val = root_mean_squared_error(true_dengrate_019_val, predicted_dengrate_019_val) / (true_dengrate_019_val.max() - true_dengrate_019_val.min())
    
        # Calculate NRMSE for both DengRate_all and DengRate_019 (training data)
        nrmse_dengrate_all_train = root_mean_squared_error(true_dengrate_all_train, predicted_dengrate_all_train) / (true_dengrate_all_train.max() - true_dengrate_all_train.min())
        nrmse_dengrate_019_train = root_mean_squared_error(true_dengrate_019_train, predicted_dengrate_019_train) / (true_dengrate_019_train.max() - true_dengrate_019_train.min())
    
        # Calculate MAE for both DengRate_all and DengRate_019 (validation data)
        mae_dengrate_all_val = mean_absolute_error(true_dengrate_all_val, predicted_dengrate_all_val)
        mae_dengrate_019_val = mean_absolute_error(true_dengrate_019_val, predicted_dengrate_019_val)
    
        # Store the results
        results.append({
            'Department': department_name,
            'NRMSE 0-19 Training': nrmse_dengrate_019_train,
            'NRMSE All Training': nrmse_dengrate_all_train,
            'NRMSE 0-19 Validation': nrmse_dengrate_019_val,
            'NRMSE All Validation': nrmse_dengrate_all_val,
            'MAE (DengRate_all) Val': mae_dengrate_all_val,
            'MAE (DengRate_019) Val': mae_dengrate_019_val
        })
    
    # Save results
    results_df = pd.DataFrame(results)
    today = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    output_file = os.path.join(config['metrics'], "Brazil", f'TFT_new_model_{today}.csv')
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")



Checking for saved models...
Loading model from: e:\minor\proj\code\saved_models\Brazil\TFT_new_model_18-03-2025-23-02-25.pt


c:\Users\Gautam\anaconda3\envs\minornew\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn(
C:\Users\Gautam\AppData\Local\Temp\ipykernel_15836\2280219853.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_glo

Results saved to e:\minor\proj\code\metrics\Brazil\TFT_new_model_18-03-2025-23-03-37.csv
